In [7]:
!pip install pyspark==3.3.1

You should consider upgrading via the '/home/aluno/Documents/curso2/venv/bin/python3.9 -m pip install --upgrade pip' command.


In [8]:
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

In [10]:
df = spark.read.json("../../datalake/twitter_datascience")

In [11]:
df.show()

+--------------------+--------------------+--------------------+------------+
|                data|            includes|                meta|extract_date|
+--------------------+--------------------+--------------------+------------+
|[{130735530154954...|{[{2020-09-19T16:...|{1595930248922763...|  2022-11-24|
|[{157414938241651...|{[{2022-09-25T21:...|{1595929788732276...|  2022-11-24|
|[{130735530154954...|{[{2020-09-19T16:...|{1595929398800269...|  2022-11-24|
|[{773127097028055...|{[{2016-09-06T11:...|{1595928938492342...|  2022-11-24|
|[{134168530765419...|{[{2020-12-23T10:...|{1595928507095617...|  2022-11-24|
|[{2965257309, 159...|{[{2015-01-07T01:...|{1595927974590701...|  2022-11-24|
|[{935446829730291...|{[{2017-11-28T09:...|{1595927859717095...|  2022-11-24|
|[{3111536505, 159...|{[{2015-03-24T20:...|{1595927547501752...|  2022-11-24|
|[{133963463913315...|{[{2020-12-17T18:...|{1595927320417689...|  2022-11-24|
|[{853309098, 1595...|{[{2012-09-29T17:...|{1595927197906358...|

In [12]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |--

In [13]:
from pyspark.sql import functions as f

In [14]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [15]:
df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{1307355301549543...|
|{1276058041389953...|
|{3111536505, 1595...|
|{1053814152798461...|
|{144592995, 15959...|
|{1053814152798461...|
|{1214696110364733...|
|{1053814152798461...|
|{240549639, 15959...|
|{1346615924535787...|
|{1574149382416515...|
|{1574149382416515...|
|{1574149382416515...|
|{1346615924535787...|
|{120993720, 15959...|
|{1357366230093545...|
|{1072579719739179...|
|{1053814152798461...|
|{1288142066082557...|
|{1163747024950452...|
+--------------------+
only showing top 20 rows



In [16]:
df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [17]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [18]:
tweet_df.show(5)

+-------------------+-------------------+--------------------+-------------------+----------+-----------+-----------+-------------+--------------------+
|          author_id|    conversation_id|          created_at|                 id|like_count|quote_count|reply_count|retweet_count|                text|
+-------------------+-------------------+--------------------+-------------------+----------+-----------+-----------+-------------+--------------------+
|1307355301549543435|1595930248922763264|2022-11-24T23:59:...|1595930248922763264|         0|          0|          0|            5|RT @usaxoom: Buy ...|
|1276058041389953024|1595930147990999045|2022-11-24T23:59:...|1595930147990999045|         0|          0|          0|            9|RT @IainLJBrown: ...|
|         3111536505|1595930101438468097|2022-11-24T23:59:...|1595930101438468097|         0|          0|          0|            7|RT @DemaiTech: Wo...|
|1053814152798461952|1595930089317109761|2022-11-24T23:59:...|1595930089317109761|

In [19]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |--

In [21]:
df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [22]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [23]:
user_df.show(5)

+--------------------+-------------------+--------------------+---------------+
|          created_at|                 id|                name|       username|
+--------------------+-------------------+--------------------+---------------+
|2020-09-19T16:26:...|1307355301549543435|           Rosia_BOT|      rosiaXbot|
|2020-06-25T07:42:...|1276058041389953024|The Data Science Bot|datasciencebot_|
|2015-03-24T20:50:...|         3111536505|            BashNano|       BashNano|
|2018-10-21T01:04:...|1053814152798461952|     MichaEL Del Rey|   MickyDelRey1|
|2010-05-16T19:15:...|          144592995|          R-bloggers|      Rbloggers|
+--------------------+-------------------+--------------------+---------------+
only showing top 5 rows



In [24]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
user_df.coalesce(1).write.mode("overwrite").json('output/user')